In [ ]:
import sys; sys.path.append("..")
from critical_ai_2026 import *


In [ ]:
import pandas as pd
df = pd.read_json("../data/emotion_classifications.json")

In [ ]:
s=df.emotion_word.value_counts()

In [ ]:
s[s>1]

In [36]:
def get_emosys_options(df):
    lines=[]
    emosys2options = {}
    gby = ["emotion_system_name", "emotion_system_year"]
    for (emosysname, emosysyear), gdf in df.groupby(gby):
        emosys = f'{emosysname} {emosysyear}'
        opts = {}
        for emocat, ggdf in gdf.groupby("emotion_category"):
            emowords = list(ggdf.emotion_word.unique())[:100]
            opts[emocat] = emowords
        
        if len(opts) > 2:
            emosys2options[emosys] = opts

    return emosys2options


In [ ]:


def emosys2prompt(emocat2words):
    system_prompt = """Read the following passage and choose which emotion categories
are represented in (depicted in, evoked by, etc) the passage.

Here are the categories from which you must choose and the 
words associated with them:

Return your answer in a JSON dictionary. The keys are the 
categories that you have chosen (each string must be the exact 
match of the category chosen), and their values are lists of strings
matching the relevant sentences from the passage that you used as
evidence for the category assignment.
"""
    lines = system_prompt.split("\n")
    for emocat, words in emocat2words.items():
        lines.append(f"* {emocat.upper()} ({', '.join(words)})")
    return "\n".join(lines)



In [64]:
emosys2options = get_emosys_options(df)
len(emosys2options)

5

In [65]:
for emocat2words in emosys2options.values():
    prompt = emosys2prompt(emocat2words)
    break

print(prompt)

Read the following passage and choose which emotion categories
are represented in (depicted in, evoked by, etc) the passage.

Here are the categories from which you must choose and the 
words associated with them:

Return your answer in a JSON dictionary. The keys are the 
categories that you have chosen (each string must be the exact 
match of the category chosen), and their values are lists of strings
matching the relevant words from the passage that you used as
evidence for the category assignment.

* AGITATION (stress, shock, tension)
* CARING (affection, empathy, friendliness, love)
* NEGATIVE AND FORCEFUL (anger, annoyance, contempt, disgust, irritation)
* NEGATIVE AND NOT IN CONTROL (anxiety, embarrassment, fear, helplessness, powerlessness, worry)
* NEGATIVE AND PASSIVE (boredom, despair, disappointment, hurt, sadness)
* NEGATIVE THOUGHTS (doubt, envy, frustration, guilt, shame)
* POSITIVE AND LIVELY (amusement, delight, elation, excitement, happiness, joy, pleasure)
* POSITIVE

In [66]:
story = generate_text("Write a 100-word story about your day today.")
print(story)


Today began with the gentle hum of rain tapping against my window, a soothing symphony that coaxed me from sleep. I brewed a cup of coffee, its warmth fuelling my morning routine. As I settled into my workspace, the world outside continued its quiet rhythms. Conversations with colleagues flickered across my screen, each pixel a bridge to connection. A crisp autumn breeze danced through the open window, rustling papers and thoughts alike. Evening descended gently, painting the sky in hues of amber and violet. As I unwind, a sense of gratitude infuses the day, a reminder of simple, cherished moments.Today began with the gentle hum of rain tapping against my window, a soothing symphony that coaxed me from sleep. I brewed a cup of coffee, its warmth fuelling my morning routine. As I settled into my workspace, the world outside continued its quiet rhythms. Conversations with colleagues flickered across my screen, each pixel a bridge to connection. A crisp autumn breeze danced through the op

In [67]:
generate_json(story, system_prompt=prompt)

```json
{
    "CARING": ["connection"],
    "QUIET POSITIVE": ["soothing", "calmness", "contentment", "relaxation", "gratitude"],
    "POSITIVE AND LIVELY": ["joy"]
}
```

{'CARING': ['connection'],
 'QUIET POSITIVE': ['soothing',
  'calmness',
  'contentment',
  'relaxation',
  'gratitude'],
 'POSITIVE AND LIVELY': ['joy']}